In [ ]:
%matplotlib notebook

import sys
import logging
import matplotlib.pyplot as plt

sys.path.append ("../DesignTool")
from smdtLibs.configFile import ConfigFile
from targets import TargetList
from smdtLibs.inOutChecker import InOutChecker
from maskLayouts import MaskLayouts, shrinkMask
from smdtLibs import utils, drawUtils

In [ ]:
try:
    %load_ext nb_black
except:
    print("Ignored extension: nb_black")
    pass

In [ ]:
logging.disable()

## Initialize configuration and read parameter file

The config file smdt.cfg contains general parameters for the application and the instrument, DEIMOS in this case.

The config file params.cfg contains parameters the user can modify through the UI. These are inherited from dsim.

In [ ]:
config_fname = "../DesignTool/smdt.cfg"
param_fname = "../DesignTool/params.cfg"

config = ConfigFile(config_fname)
config.properties["params"] = ConfigFile(param_fname)

### Get the mask layout

MaskLayouts contains the geometry of the science detector.
There are various versions of the geometry.

"deimosOrig" is the layout inherited from dsim.

In [ ]:
layout = MaskLayouts["deimos"]

### Read the targets, check what is inside and group by pcode

The pcode determines how the target is interpreted.
- \>= 0: selected
- -2: align box
- -1: guide box

In [ ]:
#input_fname = "../DeimosExamples/EvanKirby/n2419c.list"
input_fname = "../DeimosExamples/EvanKirby/LeoIa.list"
tlist = TargetList(input_fname, config=config)

# Check what is inside the mask
tlist.markInside (layout)

allTargets = tlist.targets
selectedTargets = allTargets[allTargets.selected > 0]
selectedTargets = selectedTargets[selectedTargets.pcode >= 0]

alignBoxes = allTargets[allTargets.pcode == -2]
guideBoxes = allTargets[allTargets.pcode == -1]

inTargets = allTargets[allTargets.inMask > 0]
inSelectedTargets = selectedTargets[selectedTargets.inMask > 0]
inAlignBoxes = alignBoxes[alignBoxes.inMask > 0]
inGuideBoxes = guideBoxes[guideBoxes.inMask > 0]


### What is in tlist?

In [ ]:
print (f"Number of rows:\t{len(allTargets):5d}, inside {len(inTargets):5d}")
print (f"Selected:\t{len(selectedTargets):5d}, inside {len(inSelectedTargets):5d}")
print (f"Align boxes:\t{len(alignBoxes):5d}, inside {len(inAlignBoxes):5d}")
print (f"Guide boxes:\t{len(guideBoxes):5d}, inside {len(inGuideBoxes):5d}")


In [ ]:
print("Center RA = {} hrs".format(utils.toSexagecimal(tlist.centerRADeg / 15)))
print("       DEC= {} deg".format(utils.toSexagecimal(tlist.centerDEC)))
print("       PA =  {:.3f} deg".format(tlist.positionAngle))

## Plot all targets

In [ ]:
plt.figure(figsize=(10, 6))
plt.gca().set_aspect("equal")

# Plot the mask layout
drawUtils.drawPatch(plt.gca(), layout, fc="None", ec="r")

plt.scatter(allTargets.xarcs, allTargets.yarcs, c="None", ec="y", label="Targets")
plt.scatter(
    inSelectedTargets.xarcs,
    inSelectedTargets.yarcs,
    c="None",
    ec="b",
    marker="s",
    label="Inside selected",
)
plt.legend(bbox_to_anchor=(1.30, 1.0), loc="upper right")
plt.grid()
plt.tight_layout()

### Plot only inside the mask

In [ ]:
plt.figure(figsize=(12, 5))
plt.gca().set_aspect('equal')

# Plot the mask layou
drawUtils.drawPatch(plt.gca(), layout, fc='None', ec='r')

plt.scatter (inTargets.xarcs, inTargets.yarcs,  c='None', ec='y', label='Inside')
plt.scatter (inSelectedTargets.xarcs, inSelectedTargets.yarcs, fc='None', ec='b', 
            label="Inside selected")

plt.scatter (inAlignBoxes.xarcs, inAlignBoxes.yarcs, c='None', ec='r', marker='s', label='Align boxes')
plt.scatter (inGuideBoxes.xarcs, inGuideBoxes.yarcs, c='g', ec='w', marker='s', label='Guide boxes')

plt.legend(bbox_to_anchor=(1, 1.4), loc='upper right')
plt.xlabel ("X arcsec")
plt.ylabel ("Y arcsec")

plt.legend(bbox_to_anchor=(1.20, 1.0), loc="upper right")
plt.grid()
plt.tight_layout()


In [ ]:
guideBoxes